In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset

import matplotlib.pyplot as plt
import pickle
import os
import json

## Dataloader

In [74]:
clean_dir = '/home/tk/Documents/clean/' 
mix_dir = '/home/tk/Documents/mix/' 
clean_label_dir = '/home/tk/Documents/clean_labels/' 
mix_label_dir = '/home/tk/Documents/mix_labels/' 


class MSourceDataSet(Dataset):
    
    def __init__(self, clean_dir, mix_dir, clean_label_dir, mix_label_dir):
                
        with open(clean_dir + 'clean1.json') as f:
            clean0 = torch.Tensor(json.load(f))
            
        with open(mix_dir + 'mix1.json') as f:
            mix0 = torch.Tensor(json.load(f))
        
        with open(clean_label_dir + 'clean_label1.json') as f:
            clean_label0 = torch.Tensor(json.load(f))
            

        with open(mix_label_dir + 'mix_label1.json') as f:
            mix_label0 = torch.Tensor(json.load(f))
        
        
        self.spec = torch.cat([clean0, mix0], 0)
        self.label = torch.cat([clean_label0, mix_label0], 0)
        
    def __len__(self):
        return self.spec.shape[0]

                
    def __getitem__(self, index): 

        spec = self.spec[index]
        label = self.label[index]
        return spec, label

In [78]:
trainset = MSourceDataSet(clean_dir, mix_dir, clean_label_dir, mix_label_dir)

trainloader = torch.utils.data.DataLoader(dataset = trainset,
                                                batch_size = 4,
                                                shuffle = True)

# testloader = torch.utils.data.DataLoader(dataset = testset,
#                                                batch_size = 4,
#                                                shuffle = True)

In [79]:
m = nn.MaxPool1d(3, stride =2)
input = torch.randn(20,16,50)
output = m(input)

In [83]:
(50 -1) /2 +1
(50 - 3)/ 2+1

24.5

In [80]:
print (input.shape)
print (output.shape)

torch.Size([20, 16, 50])
torch.Size([20, 16, 24])


In [76]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1025*16, 500, 3)
        self.fc1 = nn.Linear(1025*16, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)

        
    def forward(self, x):
        x = x.view(-1, 1025*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        
        return x
    
model = Net()
print (model)

Net(
  (fc1): Linear(in_features=16400, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [54]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.03)

In [77]:
model.train()
for epoch in range(10):
    running_loss = 0
    
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data

        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
#         if i % 2000 == 0:
#             print (loss.item())
        running_loss += loss.item()
        if i % 1000 == 0:
            print ('[%d, %5d] loss: %.3f' % (epoch, i, running_loss/ 1000))
            running_loss = 0
            
torch.save(model, 'FeatureNet.pkl')

[0,     0] loss: 0.009
[1,     0] loss: 0.009
[2,     0] loss: 0.009
[3,     0] loss: 0.009
[4,     0] loss: 0.009
[5,     0] loss: 0.009
[6,     0] loss: 0.009
[7,     0] loss: 0.009
[8,     0] loss: 0.009
[9,     0] loss: 0.009


/home/tk/anaconda3/lib/python3.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
